In [1]:
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd 
import numpy as np
pd.set_option("max_rows", 500)
pd.set_option("max_columns", 500)
pd.set_option('float_format', '{:f}'.format)

import dataload

import os 
import datetime
from tqdm import tqdm 
from functools import reduce

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt 
plt.style.use('seaborn')
plt.rc('font', family='AppleGothic')
plt.rc('axes', unicode_minus=False)

In [4]:
weather = pd.read_csv("weather_final2.csv", encoding="cp949")
print(weather.shape)
weather.head()

(11680, 29)


,날짜,지점번호,평균기온,최고기온,최저기온,일별강수량,1시간최대강수량,평균풍속,지역,PM10,PM25,1시간최대습도,일조시간합,최고현지기압,연,월,일,분기,요일,공휴일명,공휴일여부,주말여부,계절,체감온도,열지수,폭염여부,강수여부,year,month
0,2018-01-01,105,1.300000,5.700000,-2.100000,0.000000,0.000000,3.700000,강릉,20.066667,13.400000,25.400000,57.900000,1023.000000,2018,1,1,1,0,1월1일,1,0,3,7.502056,-39.701524,0,0,2018,1
1,2018-01-01,112,-0.300000,2.700000,-2.700000,0.000000,0.000000,1.600000,인천,37.518681,18.641758,67.200000,53.800000,1020.300000,2018,1,1,1,0,1월1일,1,0,3,9.992379,-42.924922,0,0,2018,1
2,2018-01-01,119,-1.700000,4.700000,-6.900000,0.000000,0.000000,1.000000,수원,42.782895,21.375000,84.700000,52.700000,1025.100000,2018,1,1,1,0,1월1일,1,0,3,10.136402,-45.791893,0,0,2018,1
3,2018-01-01,136,-1.000000,4.700000,-6.500000,0.000000,0.000000,2.200000,안동,39.750000,28.583333,56.400000,58.300000,1010.600000,2018,1,1,1,0,1월1일,1,0,3,8.356692,-44.375704,0,0,2018,1
4,2018-01-01,152,2.100000,6.200000,-0.400000,0.000000,0.000000,3.300000,울산,38.548193,15.433735,42.800000,58.300000,1017.100000,2018,1,1,1,0,1월1일,1,0,3,8.861429,-38.064334,0,0,2018,1


In [13]:
weather.지역.unique()

array(['강릉', '인천', '수원', '안동', '울산', '제주', '천안', '청주', '대구', '창원', '여수',
       '부산', '서울', '광주', '전주', '대전'], dtype=object)

In [14]:
weather.지역.nunique()

16

In [ ]:
'''
    - 수도권: 서울
    - 강원: 강릉
    - 경북: 대구
    - 경남: 부산
    - 전남: 광주
    - 전북: 전주 
    - 충남: 대전
    - 충북: 청주 
    - 제주: 제주
'''

In [15]:
weather[weather.지역.isin(['서울','강릉','대구','부산','광주','전주','대전','청주','제주'])].shape

(6570, 29)

In [29]:
weather2 = weather[weather.지역.isin(['서울','강릉','대구','부산','광주','전주','대전','청주','제주'])]

In [34]:
weather2.지점번호.unique()

array([105, 184, 131, 143, 159, 108, 156, 146, 133])

In [32]:
weather2.head(10)

,날짜,지점번호,평균기온,최고기온,최저기온,일별강수량,1시간최대강수량,평균풍속,지역,PM10,PM25,1시간최대습도,일조시간합,최고현지기압,연,월,일,분기,요일,공휴일명,공휴일여부,주말여부,계절,체감온도,열지수,폭염여부,강수여부,year,month
0,2018-01-01,105,1.300000,5.700000,-2.100000,0.000000,0.000000,3.700000,강릉,20.066667,13.400000,25.400000,57.900000,1023.000000,2018,1,1,1,0,1월1일,1,0,3,7.502056,-39.701524,0,0,2018,1
5,2018-01-01,184,6.400000,8.800000,4.000000,0.000000,0.000000,3.500000,제주,48.255319,21.765957,66.500000,13.100000,1027.400000,2018,1,1,1,0,1월1일,1,0,3,12.151456,-29.486495,0,0,2018,1
7,2018-01-02,105,2.500000,6.900000,-0.600000,0.000000,0.000000,2.800000,강릉,37.391304,22.869565,36.800000,53.900000,1023.300000,2018,1,2,1,1,NaN,0,0,3,10.024070,-37.263508,0,0,2018,1
9,2018-01-02,131,-0.700000,2.900000,-4.200000,0.000000,0.000000,1.200000,청주,62.548611,46.305556,84.700000,13.900000,1022.900000,2018,1,2,1,1,NaN,0,0,3,10.448620,-43.728418,0,0,2018,1
10,2018-01-02,143,1.500000,7.700000,-4.700000,0.000000,0.000000,2.300000,대구,69.537313,39.719403,77.600000,42.900000,1022.600000,2018,1,2,1,1,NaN,0,0,3,10.086958,-39.243879,0,0,2018,1
14,2018-01-03,159,2.600000,6.900000,-0.200000,0.000000,0.000000,2.800000,부산,39.297189,20.323293,56.200000,56.100000,1020.100000,2018,1,3,1,2,NaN,0,0,3,10.103983,-37.043404,0,0,2018,1
15,2018-01-04,105,0.100000,4.400000,-4.500000,0.000000,0.000000,2.600000,강릉,22.333333,14.809524,60.200000,39.500000,1025.000000,2018,1,4,1,3,NaN,0,0,3,8.468724,-42.112546,0,0,2018,1
16,2018-01-04,108,-4.700000,-0.700000,-8.700000,0.000000,0.000000,1.400000,서울,46.385027,25.381462,57.000000,41.500000,1020.400000,2018,1,4,1,3,NaN,0,0,3,7.234635,-52.095698,0,0,2018,1
17,2018-01-04,131,-3.200000,-0.900000,-6.200000,0.000000,0.000000,1.500000,청주,43.398601,33.216783,66.400000,43.900000,1023.400000,2018,1,4,1,3,NaN,0,0,3,8.097888,-48.932835,0,0,2018,1
18,2018-01-04,143,0.600000,4.800000,-3.300000,0.000000,0.000000,1.700000,대구,33.416168,21.529940,55.200000,50.300000,1022.400000,2018,1,4,1,3,NaN,0,0,3,10.464969,-41.096174,0,0,2018,1


In [23]:
population = pd.read_csv('시도별 주민등록 인구현황.csv', encoding='CP949')

In [24]:
population.head()

,시·도별(1),2018,2018.1,2018.2,2018.3,2018.4,2018.5,2018.6,2018.7,2018.8,2018.9,2019,2019.1,2019.2,2019.3,2019.4,2019.5,2019.6,2019.7,2019.8,2019.9
0,시·도별(1),총인구 (명),남 (명),여 (명),한국인_세대 (세대),한국인_계 (명),한국인_남 (명),한국인_여 (명),외국인_계 (명),외국인_남 (명),외국인_여 (명),총인구 (명),남 (명),여 (명),한국인_세대 (세대),한국인_계 (명),한국인_남 (명),한국인_여 (명),외국인_계 (명),외국인_남 (명),외국인_여 (명)
1,전국,53079513,26585715,26493798,22042947,51826059,25866129,25959930,1253454,719586,533868,53128391,26595111,26533280,22481466,51849861,25864816,25985045,1278530,730295,548235
2,서울,10049607,4910849,5138758,4263868,9765623,4773899,4991724,283984,136950,147034,10010983,4877725,5133258,4327605,9729107,4744059,4985048,281876,133666,148210
3,부산,3494019,1721327,1772692,1480468,3441453,1692421,1749032,52566,28906,23660,3466563,1704098,1762465,1497908,3413841,1675417,1738424,52722,28681,24041
4,대구,2489802,1234848,1254954,1021266,2461769,1219342,1242427,28033,15506,12527,2468222,1221693,1246529,1031251,2438031,1205286,1232745,30191,16407,13784


In [21]:
df_pop = population.loc[:,['시·도별(1)','2018','2019']]
df_population = df_pop.drop(0,axis=0).reset_index(drop=True)

In [25]:
csi = pd.read_csv("소비자심리지수.csv", encoding='cp949')

In [26]:
csi.head()

,시도,2018. 01,2018. 02,2018. 03,2018. 04,2018. 05,2018. 06,2018. 07,2018. 08,2018. 09,2018. 10,2018. 11,2018. 12,2019. 01,2019. 02,2019. 03,2019. 04,2019. 05,2019. 06,2019. 07,2019. 08,2019. 09,2019. 10,2019. 11,2019. 12,2020. 01,2020. 02,2020. 03,2020. 04,2020. 05,2020. 06,2020. 07,2020. 08,2020. 09,2020. 10,2020. 11,2020. 12
0,강릉,102.500000,98.333333,101.333333,97.333333,101.833333,99.000000,97.666667,93.000000,86.166667,86.333333,82.166667,82.666667,86.666667,88.000000,87.500000,87.000000,84.166667,86.333333,86.166667,83.166667,87.333333,86.833333,88.500000,87.666667,89.666667,86.666667,70.000000,67.166667,75.000000,79.000000,79.000000,80.000000,71.333333,78.833333,83.833333,78.333333
1,강원,103.166667,100.000000,101.666667,98.333333,101.833333,98.500000,95.000000,91.500000,86.000000,87.333333,84.500000,84.500000,87.000000,88.500000,88.166667,89.833333,86.833333,87.166667,87.000000,84.666667,88.166667,88.166667,88.333333,88.833333,90.666667,88.000000,72.833333,70.000000,75.666667,78.166667,79.166667,80.666667,72.833333,81.166667,85.000000,78.833333
2,경기,102.333333,102.666667,101.333333,100.500000,102.166667,98.833333,95.500000,91.833333,90.666667,90.500000,87.000000,87.833333,89.500000,90.666667,90.000000,92.333333,89.833333,89.500000,88.500000,86.333333,89.333333,91.166667,93.666667,93.500000,95.500000,89.666667,75.666667,71.666667,76.000000,78.500000,81.833333,83.833333,75.833333,86.500000,92.166667,84.500000
3,경남,94.666667,92.833333,93.166667,93.000000,94.666667,91.500000,88.666667,87.500000,83.666667,84.166667,80.666667,81.333333,83.000000,83.833333,84.833333,87.500000,85.666667,84.000000,84.000000,81.500000,83.500000,85.000000,87.833333,87.500000,88.666667,85.333333,71.500000,63.833333,71.333333,72.500000,76.666667,79.666667,70.500000,79.500000,85.833333,80.500000
4,광주전남,104.000000,103.666667,102.000000,102.000000,104.000000,101.500000,97.166667,93.500000,92.166667,94.333333,92.833333,91.500000,93.166667,95.500000,94.166667,94.333333,92.833333,94.000000,91.833333,88.000000,93.333333,93.833333,95.500000,96.166667,97.166667,91.000000,76.500000,73.500000,76.833333,80.166667,77.833333,82.500000,74.500000,84.666667,89.833333,83.500000


In [27]:
print(csi.shape)
csi.tail()

(14, 37)


,시도,2018. 01,2018. 02,2018. 03,2018. 04,2018. 05,2018. 06,2018. 07,2018. 08,2018. 09,2018. 10,2018. 11,2018. 12,2019. 01,2019. 02,2019. 03,2019. 04,2019. 05,2019. 06,2019. 07,2019. 08,2019. 09,2019. 10,2019. 11,2019. 12,2020. 01,2020. 02,2020. 03,2020. 04,2020. 05,2020. 06,2020. 07,2020. 08,2020. 09,2020. 10,2020. 11,2020. 12
9,인천,99.833333,97.666667,98.500000,97.166667,99.000000,97.000000,93.166667,91.333333,87.166667,88.166667,85.666667,85.166667,85.166667,87.333333,88.666667,91.166667,87.000000,87.666667,85.666667,84.833333,88.166667,88.500000,90.000000,90.333333,93.166667,85.333333,72.833333,67.666667,73.833333,75.833333,77.666667,81.333333,71.166667,83.666667,90.000000,79.666667
10,전북,103.000000,101.666667,98.833333,100.000000,101.000000,99.500000,95.666667,92.666667,92.166667,93.500000,90.666667,91.333333,90.500000,93.000000,93.333333,92.500000,91.333333,92.500000,91.333333,87.666667,91.166667,91.833333,95.000000,94.166667,97.666667,92.000000,80.000000,74.833333,80.000000,81.666667,83.666667,87.666667,79.166667,86.000000,92.166667,83.500000
11,제주,102.833333,100.000000,99.000000,100.166667,100.666667,99.833333,97.000000,92.833333,88.166667,91.000000,90.833333,88.500000,88.000000,89.000000,89.833333,92.333333,89.666667,91.833333,90.666667,88.000000,90.000000,90.500000,89.833333,90.000000,91.833333,83.500000,63.000000,70.666667,71.000000,72.666667,76.833333,80.333333,71.500000,84.333333,90.000000,79.166667
12,충북,101.000000,100.333333,99.166667,99.333333,99.333333,97.333333,93.833333,91.666667,88.666667,90.166667,88.666667,88.833333,88.833333,89.333333,90.333333,89.833333,88.500000,87.500000,87.166667,82.000000,85.666667,86.833333,87.500000,88.333333,92.166667,86.333333,73.833333,68.333333,76.833333,78.833333,78.833333,81.166667,74.333333,82.166667,86.500000,81.333333
13,서울,99.666667,98.500000,97.833333,96.833333,97.000000,96.333333,92.666667,88.666667,89.833333,90.333333,87.166667,88.333333,89.500000,92.333333,92.666667,94.000000,90.166667,90.666667,87.333333,86.333333,89.166667,90.166667,91.833333,90.833333,94.333333,88.500000,73.666667,70.000000,73.333333,77.500000,79.333333,82.166667,76.166667,84.500000,89.833333,82.166667


In [35]:
df_st = pd.read_csv('stationary_causality.csv', encoding='CP949') 

In [36]:
df_st.head()

,sm_cat,avg_ta,hm_max,max_pa,max_ta,min_ta,PM10,PM25,avg_ws,sum_ss_hr,total
0,가스온수기,None,None,None,None,None,causality,None,None,None,1
1,가자미,causality,None,None,None,None,None,None,None,None,1
2,감/홍시,None,None,None,None,None,None,None,None,None,0
3,감말랭이,None,None,None,None,None,None,None,None,causality,1
4,갓김치,causality,None,None,causality,causality,causality,None,None,None,4


In [40]:
df_st[df_st.total > 0].shape

(89, 11)

In [43]:
df_st[df_st.sm_cat == '생수']

,sm_cat,avg_ta,hm_max,max_pa,max_ta,min_ta,PM10,PM25,avg_ws,sum_ss_hr,total
81,생수,None,None,None,None,None,None,None,None,causality,1


In [45]:
def check(x) : 
    if x == 'causality' : 
        return 1 
    else :
        return 0

In [47]:
tmp = df_st[df_st.total > 0]

for col in tmp.columns : 
    tmp[col] = tmp[col].apply(lambda x : check(x))

In [56]:
tmp.drop('total', axis=1).describe()

,sm_cat,avg_ta,hm_max,max_pa,max_ta,min_ta,PM10,PM25,avg_ws,sum_ss_hr
count,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000
mean,0.000000,0.325843,0.337079,0.134831,0.382022,0.146067,0.280899,0.213483,0.146067,0.168539
std,0.000000,0.471344,0.475390,0.343479,0.488635,0.355175,0.451985,0.412088,0.355175,0.376465
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [58]:
df_st[df_st.total > 0].shape

(89, 11)

In [39]:
df_st[df_st.total > 0].sort_values('total', ascending=False)

,sm_cat,avg_ta,hm_max,max_pa,max_ta,min_ta,PM10,PM25,avg_ws,sum_ss_hr,total
96,옻/칡/쑥즙,None,causality,causality,None,causality,causality,causality,None,None,5
55,반건조고구마,None,causality,None,None,None,causality,causality,causality,causality,5
20,기능성 링클케어 화장품,causality,causality,None,causality,causality,None,None,causality,None,5
17,공기청정기,causality,None,None,causality,causality,causality,causality,None,None,5
114,태닝용 선크림,causality,None,None,causality,causality,None,causality,None,None,4
28,남성 스킨,causality,causality,None,None,None,causality,causality,None,None,4
70,뷰티용 여드름압출기,None,None,causality,causality,None,causality,causality,None,None,4
65,벽걸이형 냉온풍기,causality,None,None,causality,causality,None,None,None,causality,4
60,베이스 메이크업용 BB크림,None,causality,None,causality,None,causality,causality,None,None,4
9,견과류,causality,None,causality,causality,None,None,None,causality,None,4
